<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Advanced Skew-T Layout</h1>
<h3>Unidata Python Workshop</h3>

<div style="clear:both"></div>
</div>

<hr style="height:2px;">

<div style="float:right; width:250 px"><img src="https://unidata.github.io/MetPy/latest/_images/sphx_glr_Advanced_Sounding_001.png" alt="Example Skew-T" style="height: 500px;"></div>

### Objective
In this notebook we'll combine everything we've learned to make a nice skew-T with hodograph using `gridspec` layout.

## Obtain the data

In [ ]:
from datetime import datetime

from metpy.units import pandas_dataframe_to_unit_arrays
from siphon.simplewebservice.wyoming import WyomingUpperAir

df = WyomingUpperAir.request_data(datetime(1998, 10, 4, 0), 'OUN')
data = pandas_dataframe_to_unit_arrays(df)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from metpy.plots import SkewT, add_metpy_logo, Hodograph
import metpy.calc as mpcalc
from metpy.units import units
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np

%matplotlib inline

## Calculate parameters

In [ ]:
lcl_pressure, lcl_temperature = mpcalc.lcl(data['pressure'][0],
                                           data['temperature'][0],
                                           data['dewpoint'][0])

lfc_pressure, lfc_temperature = mpcalc.lfc(data['pressure'], data['temperature'], data['dewpoint'])

el_pressure, el_temperature = mpcalc.el(data['pressure'], data['temperature'], data['dewpoint'])

parcel_profile = mpcalc.parcel_profile(data['pressure'], data['temperature'][0], data['dewpoint'][0])

## Make the Skew-T

In [ ]:
# Create a new figure. The dimensions here give a good aspect ratio
fig = plt.figure(figsize=(9, 9))
add_metpy_logo(fig, 630, 80, size='large')

# Grid for plots
gs = gridspec.GridSpec(3, 3)
skew = SkewT(fig, rotation=45, subplot=gs[:, :2])

# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot
skew.plot(data['pressure'], data['temperature'], 'tab:red')
skew.plot(data['pressure'], data['dewpoint'], 'tab:green')
skew.plot(data['pressure'], parcel_profile, 'k')

# Mask barbs to be below 100 hPa only
mask = data['pressure'] >= 100 * units.hPa
skew.plot_barbs(data['pressure'][mask], data['u_wind'][mask], data['v_wind'][mask])
skew.ax.set_ylim(1000, 100)

# Add the relevant special lines
skew.plot_dry_adiabats()
skew.plot_moist_adiabats()
skew.plot_mixing_lines()

# Shade areas
skew.shade_cin(data['pressure'], data['temperature'], parcel_profile)
skew.shade_cape(data['pressure'], data['temperature'], parcel_profile)

# Good bounds for aspect ratio
skew.ax.set_xlim(-30, 40)

if lcl_pressure:
    skew.ax.axhline(lcl_pressure, color='black')
    
if lfc_pressure:
    skew.ax.axhline(lfc_pressure, color='tab:brown')
    
if el_pressure:
    skew.ax.axhline(el_pressure, color='tab:blue')

# Create a hodograph
agl = data['height'] - data['height'][0]
mask = agl <= 10 * units.km
boundaries = np.array([0, 1, 3, 5, 8]) * units.km
colors = ['tab:red', 'tab:green', 'tab:blue', 'tab:olive']
ax = fig.add_subplot(gs[0, -1])
h = Hodograph(ax, component_range=30.)
h.add_grid(increment=10)
h.plot_colormapped(data['u_wind'][mask], data['v_wind'][mask], agl[mask], bounds=boundaries, colors=colors)

<a href="#top">Top</a>
<hr style="height:2px;">